In [1]:
import numpy as np
from sklearn import datasets
from scipy.sparse.linalg import cg

In [2]:
bc = datasets.load_breast_cancer()
u = bc.data[:, [17, 19]]
v = bc.target
m = len(v)
softplus = lambda x : np.log(1 + np.exp(x)) if x < 100 else x
softplusD = lambda x : (1 + np.exp(-x)) ** (-1) if x > -100 else 0.0
softplusD2 = lambda x : (2 * np.cosh(x / 2)) ** (-2) if abs(x) < 100 else 0.0
def fun(x):
    return sum( (softplus(-x @ u[i]) if v[i] == 1 
                 else softplus(x @ u[i])) for i in range(m))
def gradient(x):
    return sum( (-softplusD(-x @ u[i]) * np.array(u[i]) if v[i] == 1 
                 else softplusD(x @ u[i])) for i in range(m))
def hess(x):
    return sum( (softplusD2(-x @ u[i]) * np.outer(u[i], u[i]) if v[i] == 1 
                 else softplusD2(x @ u[i]) * np.outer(u[i], u[i])) for i in range(m))

In [14]:
def trunc_newton(fun, gradient, hess, x0, gradtol = 1e-7, max_iter = 100, xbound= 1e6):
    x = x0
    for i in range(max_iter):
        print(x)
        g = gradient(x)
        if np.linalg.norm(g) <= gradtol:
            break
        if np.linalg.norm(x) > xbound:
            break
        p, _ = cg(hess(x), -g, maxiter = 10000, tol = 1e-10)
        t = 1.0
        while(fun(x + t * p) > fun(x) + 0.1 * t * np.dot(g, p)):
            t *= 0.5
        x += p 
    return x

xi = np.array([0.0, 0.0])
result = trunc_newton(fun, gradient, hess, xi)
print(result)

[0. 0.]
[  31274.04996662 -114631.76657116]
[ 3.74437757e+10 -1.37202916e+11]
[ 3.74437757e+10 -1.37202916e+11]
